### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab12_4 Stacked RNN
Lab12_3 RNN with long sequence의 코드를 재활용하며, 단지 LSTM Cell만 아래와 같이 한층을 더 쌓는 예제
![Alt text](http://i.imgur.com/c3L57jA.png)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
## Data pre-processing
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {c : i for i, c in enumerate(char_set)}
print(char_set)
print(char_dic)

sequence_length = 10

dataX = []
dataY = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:(i+sequence_length)]
    y_str = sentence[(i+1):(i+sequence_length+1)]
    
    if i % 20 == 0:
        print(i, x_str, ' ->', y_str)
        
    dataX.append([char_dic.get(x) for x in x_str])
    dataY.append([char_dic.get(y) for y in y_str])

print(dataX[:5])
print(dataY[:5])

['w', 'c', ',', 'e', 'f', 'd', 'p', 'o', '.', 'a', 'b', 'h', 'g', 'm', 'n', 's', "'", 'y', 'k', ' ', 'i', 'u', 'l', 'r', 't']
{'l': 22, 'w': 0, 'c': 1, 'd': 5, ',': 2, ' ': 19, 'i': 20, 'e': 3, 'f': 4, 'p': 6, 'u': 21, 'b': 10, 'o': 7, '.': 8, 'a': 9, 'g': 12, 'm': 13, 'n': 14, 'h': 11, 's': 15, "'": 16, 'y': 17, 't': 24, 'r': 23, 'k': 18}
0 if you wan  -> f you want
20  a ship, d  -> a ship, do
40 up people   -> p people t
60 o collect   ->  collect w
80 on't assig  -> n't assign
100 ks and wor  -> s and work
120 her teach   -> er teach t
140 ng for the  -> g for the 
160 mmensity o  -> mensity of
[[20, 4, 19, 17, 7, 21, 19, 0, 9, 14], [4, 19, 17, 7, 21, 19, 0, 9, 14, 24], [19, 17, 7, 21, 19, 0, 9, 14, 24, 19], [17, 7, 21, 19, 0, 9, 14, 24, 19, 24], [7, 21, 19, 0, 9, 14, 24, 19, 24, 7]]
[[4, 19, 17, 7, 21, 19, 0, 9, 14, 24], [19, 17, 7, 21, 19, 0, 9, 14, 24, 19], [17, 7, 21, 19, 0, 9, 14, 24, 19, 24], [7, 21, 19, 0, 9, 14, 24, 19, 24, 7], [21, 19, 0, 9, 14, 24, 19, 24, 7, 19]]


In [3]:
## Hyper-parameter setting
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
learning_rate = 0.1
batch_size = len(dataX)

In [4]:
## Graph setting
X = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
Y = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length])
X_one_hot = tf.one_hot(indices = X, depth = num_classes)
print('X_one_hot', X_one_hot)

# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, forget_bias = 0.9, state_is_tuple=True)
    return cell

# If before you were using: MultiRNNCell([BasicLSTMCell(...)] * num_layers),
# change to: MultiRNNCell([BasicLSTMCell(...) for _ in range(num_layers)])

stacked_cell = tf.contrib.rnn.MultiRNNCell(cells = [lstm_cell() for i in range(2)], state_is_tuple = True) # LSTM Cell을 2단으로 stack
print('stacked_cell_info (2-stacked)', stacked_cell.output_size, stacked_cell.state_size)

initial_states = stacked_cell.zero_state(batch_size = batch_size, dtype = tf.float32)
outputs, states = tf.nn.dynamic_rnn(cell = stacked_cell, inputs = X_one_hot, initial_state = initial_states)

# FC layer
outputs_for_fc = tf.reshape(tensor = outputs, shape = [-1, hidden_size])
print('outputs_for_fc', outputs_for_fc)
dense_weight = tf.get_variable(name = 'dense_weight', dtype = tf.float32,
                              shape = [hidden_size, num_classes],
                              initializer = tf.contrib.layers.xavier_initializer())
bias = tf.get_variable(name = 'bias', dtype = tf.float32, shape = [num_classes], initializer = tf.random_normal_initializer())
score = tf.matmul(outputs_for_fc, dense_weight) + bias

## reshape out for sequence_loss
score_for_seq_loss = tf.reshape(tensor = score, shape = [batch_size, sequence_length, num_classes])
prediction = tf.argmax(input = score_for_seq_loss, axis = -1)
print('score_for_seq_loss', score_for_seq_loss)
seq2seq_weight = tf.ones(shape = [batch_size, sequence_length])
seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits = score_for_seq_loss, targets = Y, weights = seq2seq_weight)
loss = tf.reduce_mean(seq2seq_loss)
opt = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

X_one_hot Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)
stacked_cell_info (2-stacked) 25 (LSTMStateTuple(c=25, h=25), LSTMStateTuple(c=25, h=25))
outputs_for_fc Tensor("Reshape:0", shape=(1700, 25), dtype=float32)
score_for_seq_loss Tensor("Reshape_1:0", shape=(170, 10, 25), dtype=float32)


In [5]:
## train
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(3000):
    loss_val, results, _ = sess.run(fetches = [loss, prediction, opt], feed_dict = {X : dataX, Y : dataY})
    
    if step % 300 == 0 or step == 2999:
        random_number = np.random.randint(0,len(dataX))
        print('step : {:3}, loss : {:.4f}'.format(step, loss_val))
        print(''.join([char_set[x] for x in dataX[random_number]]), '->', ''.join([char_set[hat] for hat in results[random_number]]))      

step :   0, loss : 3.5072
ou want to -> wwwwwwwwww
step : 300, loss : 0.2306
less immen -> e s immens
step : 600, loss : 0.2291
 teach the -> toach them
step : 900, loss : 0.2290
ther teach -> her toach 
step : 1200, loss : 0.2287
tasks and  -> hsks and w
step : 1500, loss : 0.2286
them tasks ->  em tasks 
step : 1800, loss : 0.2286
tasks and  -> hsks and w
step : 2100, loss : 0.2286
t rather t ->  dather te
step : 2400, loss : 0.2286
ach them t -> nh them to
step : 2700, loss : 0.2287
ensity of  -> rsity of t
step : 2999, loss : 0.2285
nsity of t -> dity of th


In [6]:
## Let's print the last char of each result to check it works
for j, result in enumerate(results):
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in result]), end='')
    else:
        print(char_set[result[-1]], end='')

t you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [7]:
## Ground truth
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."